# Monte Carlo Time Distribution Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engineerinvestor/financial-health-calculator/blob/main/examples/02_time_distribution.ipynb)

This notebook demonstrates how to run Monte Carlo simulations to understand the range of possible retirement outcomes.

In [ ]:
# Install the fundedness package
!pip install fundedness -q

In [ ]:
import numpy as np
from fundedness.models.simulation import SimulationConfig
from fundedness.models.market import MarketModel
from fundedness.simulate import run_simulation
from fundedness.viz.fan_chart import create_fan_chart, create_spending_fan_chart
from fundedness.viz.survival import create_survival_curve, create_dual_survival_chart
from fundedness.viz.histogram import create_time_distribution_histogram

## 1. Set Up Simulation Parameters

In [ ]:
# Simulation inputs
initial_wealth = 1_000_000  # $1M starting portfolio
annual_spending = 40_000    # $40k/year (4% rule)
spending_floor = 30_000     # Essential spending floor
stock_allocation = 0.60     # 60% stocks / 40% bonds

# Market assumptions
market_model = MarketModel(
    stock_return=0.05,      # 5% real return
    bond_return=0.015,      # 1.5% real return
    stock_volatility=0.16,  # 16% annual volatility
    inflation_mean=0.025,   # 2.5% inflation
)

# Simulation config
config = SimulationConfig(
    n_simulations=10_000,
    n_years=40,
    random_seed=42,
    market_model=market_model,
)

print(f"Portfolio: ${initial_wealth:,}")
print(f"Spending: ${annual_spending:,}/year ({annual_spending/initial_wealth:.1%})")
print(f"Allocation: {stock_allocation:.0%} stocks")

## 2. Run the Simulation

In [ ]:
%%time
# Run Monte Carlo simulation
result = run_simulation(
    initial_wealth=initial_wealth,
    annual_spending=annual_spending,
    config=config,
    stock_weight=stock_allocation,
    spending_floor=spending_floor,
)

In [ ]:
# Key metrics
print(f"Success Rate: {result.success_rate:.1%}")
print(f"Floor Breach Rate: {result.floor_breach_rate:.1%}")
print(f"Median Terminal Wealth: ${result.median_terminal_wealth:,.0f}")
print(f"Mean Terminal Wealth: ${result.mean_terminal_wealth:,.0f}")

## 3. Wealth Projection Fan Chart

This chart shows the range of possible portfolio values over time (P10, P25, P50, P75, P90):

In [ ]:
years = np.arange(1, config.n_years + 1)

fig = create_fan_chart(
    years=years,
    percentiles=result.wealth_percentiles,
    title="Portfolio Value Projection",
    y_label="Portfolio Value ($)",
)
fig.show()

## 4. Survival Probability

What's the probability of your portfolio lasting each year?

In [ ]:
survival_prob = result.get_survival_probability()
floor_survival_prob = result.get_floor_survival_probability()

fig = create_survival_curve(
    years=years,
    survival_prob=survival_prob,
    floor_survival_prob=floor_survival_prob,
    title="Portfolio Survival Probability",
    threshold_years=[10, 20, 30],
)
fig.show()

In [ ]:
# Key survival metrics
print("Survival Probability:")
for y in [10, 20, 30, 40]:
    if y <= len(survival_prob):
        print(f"  {y} years: {survival_prob[y-1]:.1%}")

## 5. Risk Timeline

See how risk accumulates over time:

In [ ]:
ruin_prob = 1 - survival_prob
floor_breach_prob = 1 - floor_survival_prob

fig = create_dual_survival_chart(
    years=years,
    ruin_prob=ruin_prob,
    floor_breach_prob=floor_breach_prob,
    title="Cumulative Risk Over Time",
)
fig.show()

## 6. Time to Ruin Distribution

For paths that do run out of money, when does it happen?

In [ ]:
fig = create_time_distribution_histogram(
    time_to_event=result.time_to_ruin,
    event_name="Ruin",
    planning_horizon=config.n_years,
    percentiles_to_show=[10, 25, 50],
)
fig.show()

## 7. Sensitivity to Withdrawal Rate

Let's see how different withdrawal rates affect success:

In [ ]:
withdrawal_rates = [0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06]
success_rates = []

for rate in withdrawal_rates:
    spending = initial_wealth * rate
    result_i = run_simulation(
        initial_wealth=initial_wealth,
        annual_spending=spending,
        config=SimulationConfig(n_simulations=5000, n_years=30, random_seed=42),
        stock_weight=stock_allocation,
    )
    success_rates.append(result_i.success_rate)
    print(f"{rate:.1%} withdrawal rate: {result_i.success_rate:.1%} success")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=[r * 100 for r in withdrawal_rates],
    y=[s * 100 for s in success_rates],
    mode='lines+markers',
    line=dict(width=3, color='#3498db'),
    marker=dict(size=10),
))

fig.add_hline(y=90, line_dash="dash", line_color="green", 
              annotation_text="90% Target")

fig.update_layout(
    title="Success Rate vs Withdrawal Rate",
    xaxis_title="Withdrawal Rate (%)",
    yaxis_title="Success Rate (%)",
    template="plotly_white",
)
fig.show()

## 8. Sensitivity to Asset Allocation

In [ ]:
allocations = [0.2, 0.4, 0.6, 0.8, 1.0]
success_by_alloc = []

for alloc in allocations:
    result_i = run_simulation(
        initial_wealth=initial_wealth,
        annual_spending=40_000,
        config=SimulationConfig(n_simulations=5000, n_years=30, random_seed=42),
        stock_weight=alloc,
    )
    success_by_alloc.append(result_i.success_rate)
    print(f"{alloc:.0%} stocks: {result_i.success_rate:.1%} success, median terminal = ${result_i.median_terminal_wealth:,.0f}")

## Summary

Key insights from this analysis:

1. **Monte Carlo shows the range** of possible outcomes, not just averages
2. **Success rate decreases** with higher withdrawal rates
3. **Time to ruin** helps understand when failures occur
4. **Asset allocation** affects both returns and volatility

Use these tools to find a sustainable withdrawal rate for your situation!